In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import tensorflow as tf
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import WhiteKernel


In [ ]:
def gaussian(inp,mean,var):
    std=np.sqrt(var)
    value=(1/(np.sqrt(2*np.pi*var)))*np.exp((-1*(mean-inp)**2)/(2*var))
    return value

In [ ]:
def membership(inp,mean_l,mean_m,mean_h,var=0.8):#l,h are the base width of the right triangle.m is the half base width of the isoceles
    ul,um,uh=0,0,0
    ul=gaussian(inp,mean_l,var)
    um=gaussian(inp,mean_m,var)
    uh=gaussian(inp,mean_h,var)
    ULMH=np.array([[ul,um,uh]])
    return ULMH.reshape(1,3) # matrix of dimensions 1,3

In [ ]:
def relu(out):
    if out>0:
        return out
    else:
        return 0.1*out # leaky relu 

In [ ]:
class node():
    
    def __init__(self):
        self.LMH=np.random.uniform(0,300, (2,3))# generates 2*3 matrix of random numbers between 0 to 0.5
        self.ab=np.random.randint(1,4,(9,2))#9*2 matrix of random numbers between 1 to 4 excludiing 4
        self.LMH_update=np.zeros((2,3))#update gonna be used during backprop
        self.ab_update=np.zeros((9,2))#update gonna be used during backprop
        self.LMH[0][0]=0
        self.LMH[0][1]=0.5
        self.LMH[0][2]=1
        self.LMH[1][0]=0
        self.LMH[1][1]=0.5
        self.LMH[1][2]=1
    def membership_class(self,inp1,inp2):
        self.inp1=inp1
        self.inp2=inp2
        self.U1=membership(self.inp1,self.LMH[0][0],self.LMH[0][1],self.LMH[0][2])# matrix of dimensions 1,3
        self.U2=membership(self.inp2,self.LMH[1][0],self.LMH[1][1],self.LMH[1][2])# matrix of dimensions 1,3   
        
        
    def membership_class_node1(self,inp1,inp2):
        self.inp1=inp1
        self.inp2=inp2
        self.LMH[0][0]=20
        self.LMH[0][1]=140
        self.LMH[0][2]=260
        self.LMH[1][0]=0
        self.LMH[1][1]=0.5
        self.LMH[1][2]=1
        self.U1=membership(self.inp1,self.LMH[0][0],self.LMH[0][1],self.LMH[0][2],var=1000)# matrix of dimensions 1,3
        self.U2=membership(self.inp2,self.LMH[1][0],self.LMH[1][1],self.LMH[1][2])# matrix of dimensions 1,3       
        
    def forward_prop(self):
        self.inp=np.array([[self.inp1],[self.inp2]])
        #print(self.inp)
        self.inp=self.inp.reshape(2,1) #matrix of dimensions 2*1
        
        self.k=np.dot(self.U1.T,self.U2)#k is a 3*3 matrix
        
        self.ksum=np.sum(self.k)
        self.ksum=float(self.ksum)
        if self.ksum==0:
            self.ksum=1
            
        self.k_norm=self.k/self.ksum #k_norm is normalised
        
        self.w=self.k_norm.reshape(1,9)#self is a matrix of dimensions 1*9
        
        self.y=np.dot(self.ab,self.inp) #y is a matrix of dimensions 9*1
        
        self.out_before_relu=np.dot(self.w,self.y) #out is a matrix of dimension 1*1
        
        self.out_before_relu=float(self.out_before_relu)# typecasts matrix of size 1*1 to float
        
        self.out=relu(self.out_before_relu)# relu activation function is used
        #print('LMH',self.LMH)
    
    def backprop_to_prev_node (self,learning_rate,last):
        if self.out_before_relu>0:
            self.relu_drvt=1
        else:
            self.relu_drvt=0.1 # derivative of relu
            
        self.to_prev=learning_rate*last*self.relu_drvt*(np.sum(self.ab[:,0]*self.w.T)) 
        
        self.to_prev=float(self.to_prev)
        
        
        
        
    def backprop_ab(self,learning_rate,last):
        self.ab_update=self.ab_update+learning_rate*last*self.relu_drvt*self.w.reshape(9,1)*self.inp.T
        
        
        
        
    def backprop_LMH(self,learning_rate,last,var=0.8): # backprop with gaussian membership function. var is variance of gaussian
        
        self.d=(self.ksum-self.k)/(self.ksum*self.ksum) # d is a 3*3 matrix
        
        self.dummy=self.y.reshape(3,3)*self.d*self.k #3*3 matrix
        
        self.dummy1=self.dummy.sum(axis=1)# sums along rows. 3*1 matrix
        self.dummy2=self.dummy.sum(axis=0)# sums along columns . 1*3 matrix
        
        self.drvt=(self.LMH-self.inp)/(2*np.sqrt(var)) # LMH is 2*3 inp is 2*1 drvt is 2*3 matrix 
        
        self.dummy1=self.dummy1.T*self.drvt[0,:] # dummy1.T is 1*3 drvt[0,:] is 1*3 dummy1 is 1*3 
        self.dummy2=self.dummy2*self.drvt[1,:] # dummy2 is 1*3 drvt[1,:] is 1*3 dummy2 is 1*3 
        
        self.LMH_update[0,:]=self.LMH_update[0,:]+learning_rate*last*self.relu_drvt*self.dummy1 # 1*3 
        self.LMH_update[1,:]=self.LMH_update[1,:]+learning_rate*last*self.relu_drvt*self.dummy2 # 1*3 
        
        
        
    def update(self):
        self.ab=self.ab+self.ab_update
        self.LMH=self.LMH+self.LMH_update
        
    def reset(self):
        self.LMH_update=np.zeros((2,3))
        self.ab_update=np.zeros((9,2))
        
        

In [ ]:
class layer():
    def __init__(self):
        self.a=0
            
            
    def initalise(self,n,n_inp): # n is the number of nodes in a layer. n_inp are the number of inputs to that node.
        self.n=n
        self.n_inp=n_inp
        self.weights=np.random.uniform(-1,1, (n_inp,n))#random numbers between -1 to 1 generated, n_inp*n matrix
        self.bias=np.random.uniform(-1,1, (1,n))# 1*n matrix
        self.activate=np.zeros((1,n))
        self.d_activation_relu=np.zeros((1,n))
    
    def forwardpass(self,inp) :
        self.inp=inp
        self.forward=np.dot(self.inp,self.weights)+self.bias
        
        for i in range(self.n):
            self.activate[0][i]=relu(self.forward[0][i])
            
    def drv_activation_relu(self):
        for i in range(self.n):
            if self.activate[0][i] >=0:
                self.d_activation_relu[0][i]=1
            else:
                self.d_activation_relu[0][i]=-0.1
                
                
    def to_prev_layer(self,from_next,d_act_pre):# from_next is output from neurons ahead to this layer (backprop)
        self.from_next=from_next# n*1  #d_act_pre is the derivative of the activation of the previous neuron
        self.d_act_pre=d_act_pre #1*n
        
        
        self.w_pre=np.dot(self.weights,self.from_next*self.d_act_pre.reshape(self.n,1))# n_inp*1
        self.to_pre=self.w_pre #n_inp*1


    def to_prev_layer1(self,from_next,d_act_pre):# from_next is output from neurons ahead to this layer (backprop)
        self.from_next=from_next# n*1  #d_act_pre is the derivative of the activation of the previous neuron
        self.d_act_pre=d_act_pre #1*n_inp
        self.w_pre=self.weights*self.from_next*self.d_act_pre# n_inp*1
        self.to_pre=self.w_pre #n_inp*1     
        
     
    
    def backprop_weights(self,lr,from_next,d_act_pre): #d_act_pre is 1*n, self.inp is 1*n_inp,from_next is n*1
        self.update_weights=np.zeros((self.n_inp,self.n)) #n_inp*n
        self.update_bias=np.zeros((1,self.n)) # 1*n
        self.dummy_weights=np.ones((self.n_inp,self.n)) #n_inp*n
        self.dummy_bias=np.ones((1,self.n)) #1*n
        
        self.update_weights=self.update_weights+lr*(self.dummy_weights*self.inp.reshape(self.n_inp,1))*from_next.reshape(1,self.n)*d_act_pre
        self.update_bias=self.update_bias+lr*self.dummy_bias*d_act_pre*from_next.reshape(1,self.n)
        
    def backprop_weights1(self,lr,from_next,d_act_pre): #d_act_pre is 1*n, self.inp is 1*n_inp,from_next is n*1
        self.update_weights=np.zeros((self.n_inp,self.n)) #n_inp*n
        self.update_bias=np.zeros((1,self.n)) # 1*n
        self.dummy_weights=np.ones((self.n_inp,self.n)) #n_inp*n
        self.dummy_bias=np.ones((1,self.n)) #1*n
        
        self.update_weights=self.update_weights+lr*(self.dummy_weights*self.inp.reshape(self.n_inp,1))*from_next*d_act_pre
        self.update_bias=self.update_bias+lr*self.dummy_bias*d_act_pre*from_next
        
    def update(self):
        self.weights=self.weights+self.update_weights
        self.bias=self.bias+self.update_bias
    
    
  
        

In [ ]:
df = pd.read_csv("train_FD001.txt", sep=" ",header=None)
train=df
train.drop([26,27],axis=1,inplace=True) #drop columns as both of them contain NaN values
train.drop([4,5,9,10,14,20,22,23],axis=1,inplace=True) # columns with constant values
train.columns=['ID','Cycles','OS1','OS2','S1','S2','S3','S4','S5','S6','S7','S8','S9','S10','S11','S12','S13','S14']
EOL=[]
for i in train['ID']:
        EOL.append( ((train[train['ID'] == i]["Cycles"]).values)[-1])
train["EOL"]=EOL
train["RULf"] = train["Cycles"].div(train["EOL"])
train=train.drop(columns=['ID','EOL'])

In [ ]:
to_be_normalised=train[['S1','S2','S3','S4','S5','S6','S7','S8','S9','S10','S11','S12','S13','S14']]
data_min=to_be_normalised.min()
data_max=to_be_normalised.max()
data_min=data_min.to_numpy()#converting pandas to numpy array of dimension 14*1
data_max=data_max.to_numpy()
data_min=np.reshape(data_min,(1,14))#reshaping 14*1 to 1*14
data_max=np.reshape(data_max,(1,14))
normalised=(to_be_normalised-to_be_normalised.min())/(to_be_normalised.max()-to_be_normalised.min())


In [ ]:
train[['S1','S2','S3','S4','S5','S6','S7','S8','S9','S10','S11','S12','S13','S14']]=normalised[['S1','S2','S3','S4','S5','S6','S7','S8','S9','S10','S11','S12','S13','S14']]
train["Cycles"]=train["Cycles"]/500
train = train.sample(frac=1).reset_index(drop=True)
target = train.pop('RULf')
train_matrix=train.to_numpy()
tar=target.to_numpy()


In [ ]:
node1=node()
node2=node()
node3=node()
node4=node()
node5=node()
node6=node()
node7=node()
layer1=layer()
layer2=layer()
layer3=layer()

In [ ]:
print (train_matrix)
train_matrix.shape

In [ ]:
layer1.initalise(8,10) #O/P,I/P
layer2.initalise(6,8)
layer3.initalise(1,6)

In [ ]:
b=np.zeros((1,10))
mse=0
cycle=np.zeros((20631,1))
Yout=np.zeros((20631,1))
Yerr=np.zeros((20631,1))
lr=0.01
for i in range(0,2063):
    for j in range(0,10):
        inp=train_matrix[i*10+j]
        cycle[i*10+j]=i*10+j
    
        print(i*10+j)
        node1.membership_class(inp[3],inp[4])
        node1.forward_prop()
        node2.membership_class(inp[5],inp[6])
        node2.forward_prop()
        node3.membership_class(inp[7],inp[8])
        node3.forward_prop()
        node4.membership_class(inp[9],inp[10])
        node4.forward_prop()
        node5.membership_class(inp[11],inp[12])
        node5.forward_prop()
        node6.membership_class(inp[13],inp[14])
        node6.forward_prop()
        node7.membership_class(inp[15],inp[16])
        node7.forward_prop()
        
        b=np.array([[inp[0],inp[1],inp[2],node1.out,node2.out,node3.out,node4.out,node5.out,node6.out,node7.out]])
        
        
      
        layer1.forwardpass(b)
        layer2.forwardpass(layer1.activate)
        layer3.forwardpass(layer2.activate)
        #print(layer1.activate)
        
        Yout[i*10+j]=float(layer3.forward)
        Yerr[i*10+j]=(float(layer3.forward)-tar[i*10+j])**2
        
        layer1.drv_activation_relu()
        layer2.drv_activation_relu()
        layer3.drv_activation_relu
        
        
        err=2*(tar[i*10+j]-float(layer3.forward))
        
        z=np.array([[1]])
   
        layer3.to_prev_layer1(err,1)
        layer3.backprop_weights1(lr,err,1)
        
        layer2.to_prev_layer(layer3.to_pre,layer2.d_activation_relu)
        layer2.backprop_weights(lr,layer3.to_pre,layer2.d_activation_relu)
        
        
        layer1.to_prev_layer(layer2.to_pre,layer1.d_activation_relu)
        layer1.backprop_weights(lr,layer2.to_pre,layer1.d_activation_relu)
        
        
        #print(layer1.to_pre)
        node1.backprop_to_prev_node(lr,layer1.to_pre[3][0])
        node1.backprop_ab(lr,layer1.to_pre[3][0])
        node1.backprop_LMH(lr,layer1.to_pre[3][0])
        
        
        node2.backprop_to_prev_node(lr,layer1.to_pre[4][0])
        node2.backprop_ab(lr,layer1.to_pre[4][0])
        node2.backprop_LMH(lr,layer1.to_pre[4][0])
        
        
        node3.backprop_to_prev_node(lr,layer1.to_pre[5][0])
        node3.backprop_ab(lr,layer1.to_pre[5][0])
        node3.backprop_LMH(lr,layer1.to_pre[5][0])
        
        node4.backprop_to_prev_node(lr,layer1.to_pre[6][0])
        node4.backprop_ab(lr,layer1.to_pre[6][0])
        node4.backprop_LMH(lr,layer1.to_pre[6][0])
        
        node5.backprop_to_prev_node(lr,layer1.to_pre[7][0])
        node5.backprop_ab(lr,layer1.to_pre[7][0])
        node5.backprop_LMH(lr,layer1.to_pre[7][0])
        
        node6.backprop_to_prev_node(lr,layer1.to_pre[8][0])
        node6.backprop_ab(lr,layer1.to_pre[8][0])
        node6.backprop_LMH(lr,layer1.to_pre[8][0])
        
        node7.backprop_to_prev_node(lr,layer1.to_pre[9][0])
        node7.backprop_ab(lr,layer1.to_pre[9][0])
        node7.backprop_LMH(lr,layer1.to_pre[9][0])
        
        print("out", layer3.forward,"tar",tar[i*10+j],"error",tar[i*10+j]-layer3.forward)
       # print(layer1.to_pre.shape)
        #print(layer2.to_pre.shape)
        
        mse=mse+(tar[i*10+j]-layer3.forward)**2
        layer3.update()
        layer2.update()
        layer1.update()
        node1.update()
        node1.reset()
        node2.update()
        node2.reset()
        node3.update()
        node3.reset()
        node4.update()
        node4.reset()
        node5.update()
        node5.reset()
        node6.update()
        node6.reset()
        node7.update()
        node7.reset()
        

        

In [ ]:
print(mse)
print(mse/20630) #all input variables present+ 2 hidden layers

In [ ]:
plt.axis([0,20630 , 0, 1])
plt.scatter(cycle,Yerr)
plt.show()

In [ ]:
print("lmh1",node1.LMH)

In [ ]:
plt.scatter(tar[20000:],Yout[20000:])
plt.show()

In [ ]:
plt.scatter(tar[20000:],Yerr[20000:])
plt.show()

In [ ]:
df1 = pd.read_csv("test_FD001.txt", sep=" ",header=None)
test=df1
test.drop([26,27],axis=1,inplace=True) #drop columns as both of them contain NaN values
test.drop([4,5,9,10,14,20,22,23],axis=1,inplace=True) # columns with constant values
test.columns=['ID','Cycles','OS1','OS2','S1','S2','S3','S4','S5','S6','S7','S8','S9','S10','S11','S12','S13','S14']

In [ ]:
to_be_normalised1=test[['S1','S2','S3','S4','S5','S6','S7','S8','S9','S10','S11','S12','S13','S14']]

#min max normalisation
normalised1=(to_be_normalised1-data_min)/(data_max-data_min)
test[['S1','S2','S3','S4','S5','S6','S7','S8','S9','S10','S11','S12','S13','S14']]=normalised1[['S1','S2','S3','S4','S5','S6','S7','S8','S9','S10','S11','S12','S13','S14']]
test["Cycles"]=test["Cycles"]/500

In [ ]:
def pred_test(a):
    b=np.zeros((1,10))
    test_pred=test.loc[test['ID'] == a]
    inp_test_pred=test_pred[['Cycles','OS1','OS2','S1','S2','S3','S4','S5','S6','S7','S8','S9','S10','S11','S12','S13','S14']]
    test_matrix=inp_test_pred.to_numpy()
    Y_pred=[0]*len(test_matrix)
    for j in range(0,len(test_matrix)):
        inp=test_matrix[j]
        #print(test_matrix[j])
        #print(j)
        node1.membership_class(inp[3],inp[4])
        node1.forward_prop()
        node2.membership_class(inp[5],inp[6])
        node2.forward_prop()
        node3.membership_class(inp[7],inp[8])
        node3.forward_prop()
        node4.membership_class(inp[9],inp[10])
        node4.forward_prop()
        node5.membership_class(inp[11],inp[12])
        node5.forward_prop()
        node6.membership_class(inp[13],inp[14])
        node6.forward_prop()
        node7.membership_class(inp[15],inp[16])
        node7.forward_prop()
        
        b=np.array([[inp[0],inp[1],inp[2],node1.out,node2.out,node3.out,node4.out,node5.out,node6.out,node7.out]])
        
        
      
        layer1.forwardpass(b)
        layer2.forwardpass(layer1.activate)
        layer3.forwardpass(layer2.activate)
        #Y_pred[j]=float(1-layer3.forward)
        Y_pred[j]=float(layer3.forward)
        #Y_pred=1-np.array(Y_pred)
        #Y_pred=np.array(Y_pred)
        #print(Y_pred.shape)
        #print(Y_pred)
    
    
    #plt.plot(test_pred['Cycles']*500,Y_pred,c='red')
    #plt.show()
    Y_pred=np.array(Y_pred)
    #print(Y_pred)
    

    return Y_pred ,test_pred['Cycles'].to_numpy().reshape(-1,1)*500,test_pred['Cycles'].max()*500
    

In [ ]:
df2 = pd.read_csv("test_FD001.txt", sep=" ",header=None)
test2=df2
test2.drop([26,27],axis=1,inplace=True) #drop columns as both of them contain NaN values
test2.drop([4,5,9,10,14,20,22,23],axis=1,inplace=True) # columns with constant values
test2.columns=['ID','Cycles','OS1','OS2','S1','S2','S3','S4','S5','S6','S7','S8','S9','S10','S11','S12','S13','S14']
Truncate=[]
for i in test2['ID']:
        
        Truncate.append( ((test2[test2['ID'] == i]["Cycles"]).values)[-1])
new_arr = []
for i in range(len(Truncate)):
    if i == 0 or Truncate[i] != Truncate[i-1]:
        new_arr.append(Truncate[i])
        
print(Truncate)
print(new_arr)
np.shape(new_arr)

In [ ]:
rul = pd.read_csv("RUL_FD001.txt", sep=" ",header=None)
rul[1]=new_arr
rul[2]=rul[0]+rul[1]

In [ ]:
X_test = np.linspace(1, 350, 350).reshape(-1,1)

In [ ]:
from sklearn.gaussian_process.kernels import RBF

rbf = RBF(length_scale=2.7) #long_term_trend_kernel

In [ ]:
from sklearn.gaussian_process.kernels import DotProduct

# Set the hyperparameters
sigma_0 = 1
c = 1.0#x.y+c
p = 1#(x.y+c)^p

# Create the kernel
lin =  DotProduct(sigma_0=sigma_0, sigma_0_bounds=(1e-5, np.inf))


In [ ]:
from sklearn.gaussian_process.kernels import RationalQuadratic

rq =  RationalQuadratic(length_scale=1, alpha=1) #irregularities_kernel

In [ ]:
from sklearn.gaussian_process.kernels import WhiteKernel

nk = 0.2**2 * RBF(length_scale=0.1) + WhiteKernel(
    noise_level=0.05**2, noise_level_bounds=(1e-5, 1e5)
)#noise kernel 


In [ ]:
def RUL(z):
    results=np.zeros((100,1))
    truncate=np.zeros((100,1))

    Y_train,X_train,dummy=pred_test(z)

    Y_train=1-Y_train
#Y_train=Y_train*100
    gaussian_process = GaussianProcessRegressor(kernel, normalize_y=False)
    Y_mean=Y_train.mean()

    X_train=X_train.reshape(-1,1)
    gaussian_process.fit(X_train,Y_train)
    
    mean_y_pred, std_y_pred = gaussian_process.predict(X_test, return_std=True)
    #mean_y_pred += Y_mean
    plt.plot(X_train, Y_train, color="black", linestyle="dashed", label="Measurements")
    plt.plot(X_test, mean_y_pred, color="tab:blue", alpha=0.4, label="Gaussian process")
    plt.fill_between(
        X_test.ravel(),
        mean_y_pred - std_y_pred,
        mean_y_pred + std_y_pred,
        color="tab:blue",
        alpha=0.2,
    )
    for i in range(349):
        if (mean_y_pred[i]*mean_y_pred[i+1]<0):
            print(X_test[i])
            results[j-1]=X_test[i]
            break

In [ ]:
kernel=rbf+nk+lin+rq

In [ ]:
results=np.zeros((100,1))
truncate=np.zeros((100,1))
for j in range(1,101):
    print(j)
    X_test = np.linspace(1, 400, 400).reshape(-1,1)
    Y_train,X_train,truncate[j-1]=pred_test(j)

    Y_train=1-Y_train
    #Y_train=Y_train*100
    gaussian_process = GaussianProcessRegressor(kernel, normalize_y=False)
    Y_mean=Y_train.mean()
    X_train=X_train.reshape(-1,1)
    gaussian_process.fit(X_train,Y_train)
    mean_y_pred, std_y_pred = gaussian_process.predict(X_test, return_std=True)
    plt.plot(X_train, Y_train, color="black", linestyle="dashed", label="Measurements")
    plt.plot(X_test, mean_y_pred, color="tab:blue", alpha=0.4, label="Gaussian process")
    plt.fill_between(
    X_test.ravel(),
    mean_y_pred - std_y_pred,
    mean_y_pred + std_y_pred,
    color="tab:blue",
    alpha=0.2,
    )
    plt.xlabel("Cycles")
    plt.ylabel("Engine Health")

    plt.show()
    for i in range(349):
        if (mean_y_pred[i]*mean_y_pred[i+1]<0):
            print("result",X_test[i])
            print("actual",rul[2][j-1])
            
            results[j-1]=X_test[i]
            break
    for i in range(349):
        if (mean_y_pred[i]- std_y_pred[i]*mean_y_pred[i+1]- std_y_pred[i+1]<0):
            print("lower band",X_test[i])
            results[j-1]=X_test[i]
            break
            
    for i in range(349):
        if (mean_y_pred[i]+ std_y_pred[i]*mean_y_pred[i+1]+ std_y_pred[i+1]<0):
            print("upper band",X_test[i])
            results[j-1]=X_test[i]
            break

In [ ]:
rul1 = pd.read_csv("RUL_FD001.txt", sep=" ",header=None)

In [ ]:
count=0
for i in results:
    #print(i)
    if i==0:
        count=count+1
print(count)        

In [ ]:
rul1[1]=results
rul1[2]=truncate
rul1[3]=rul1[0]+rul1[2]#final RUL
rul1[4]=rul1[3]-rul1[1]#error

In [ ]:
rul1 = rul1[(rul1[[1]] != 0).all(axis=1)]


In [ ]:
print(rul1)

In [ ]:
error=rul1.pop(4)


In [ ]:
er=error.to_numpy()


In [ ]:
ersq=0
for i in range(100):
    print(i+1,er[i])
    ersq=ersq+(er[i])**2

In [ ]:
mse=(ersq/i)**0.5

In [ ]:
print(mse)